<a href="https://colab.research.google.com/github/smeisner10/f1_ml/blob/master/F1_Classification_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, precision_score
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn import svm
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.neural_network import MLPClassifier, MLPRegressor

np.set_printoptions(precision=4)

In [ ]:
# mount drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/F1_ML/f1_ml-master/scraping/
data = pd.read_csv('final_df.csv')

/content/drive/MyDrive/F1_ML/f1_ml-master/scraping


# Regression

In [ ]:
comparison_dict ={'model':[],
                  'params': [],
                  'score': []}

In [ ]:
df = data.copy()
df = df.drop(['qualifying_time'], axis = 1)

train = df[df.season <2022]

X_train = train.drop(['driver', 'podium'], axis = 1)
y_train = train.podium

scaler = StandardScaler()

X_train = pd.DataFrame(scaler.fit_transform(X_train), columns = X_train.columns)

In [ ]:
 def score_regression(model):
    score = 0
    preds = pd.DataFrame()
    for circuit in df[df.season == 2022]['round'].unique():
        test = df[(df.season == 2022) & (df['round'] == circuit)]
        X_test = test.drop(['driver', 'podium'], axis = 1)
        y_test = test.podium

        #scaling
        X_test = pd.DataFrame(scaler.transform(X_test), columns = X_test.columns)

        # make predictions
        prediction_df = pd.DataFrame(model.predict(X_test), columns = ['results'])
        prediction_df['podium'] = y_test.reset_index(drop = True)
        prediction_df['actual'] = prediction_df.podium.map(lambda x: 1 if x == 1 else 0)
        prediction_df.sort_values('results', ascending = True, inplace = True)
        prediction_df.reset_index(inplace = True, drop = True)
        prediction_df['predicted'] = prediction_df.index
        prediction_df['predicted'] = prediction_df.predicted.map(lambda x: 1 if x == 0 else 0)
        prediction_df['round'] = prediction_df.predicted.map(lambda x: circuit if True else circuit)
        preds = preds.append(prediction_df)

        score += precision_score(prediction_df.actual, prediction_df.predicted)

    model_score = score / len(df[df.season == 2022]['round'].unique())
    return model_score, preds

In [ ]:
# Linear Regression

model = LinearRegression(fit_intercept = 'True') # can change to false
model.fit(X_train, y_train)
        
scored_regression = score_regression(model)
model_score = scored_regression[0]
preds = scored_regression[1]
            
comparison_dict['model'].append('linear_regression')
comparison_dict['params'].append(True) # corresponding with above
comparison_dict['score'].append(model_score)

In [ ]:
comparison_df = pd.DataFrame(comparison_dict)
comparison_df

,model,params,score
0,linear_regression,True,0.590909
1,linear_regression,False,0.590909
2,svm_regressor,False,0.590909
3,linear_regression,True,0.684211
4,linear_regression,False,0.684211


In [ ]:
## Support Vector Machines

model = svm.SVR(gamma = .01, C = 1, kernel = 'linear')
model.fit(X_train, y_train)

score_svm = score_regression(model)
model_score_svm = score_svm[0]
preds_svm = score_svm[1]

comparison_dict['model'].append('svm_regressor')
comparison_dict['params'].append(model_params)
comparison_dict['score'].append(model_score)

2022 odds: https://www.vegasinsider.com/auto-racing/odds/f1/

In [ ]:
preds_svm

,results,podium,actual,predicted,round
1,2.978029,19,0,1,1
0,3.200136,1,1,0,1
0,3.424540,2,0,0,1
0,4.114579,18,0,0,1
0,5.160251,3,0,0,1
...,...,...,...,...,...
0,14.549459,11,0,0,22
0,15.747258,14,0,0,22
0,15.768089,17,0,0,22
0,16.151361,13,0,0,22


In [ ]:
final_results = preds_svm.merge(preds, on = ["round", "podium"])
final_results


,results_x,podium,actual_x,predicted_x,round,results_y,actual_y,predicted_y
0,2.978029,19,0,1,1,6.228506,0,0
1,3.200136,1,1,0,1,6.211187,1,1
2,3.424540,2,0,0,1,6.302891,0,0
3,4.114579,18,0,0,1,6.217155,0,0
4,5.160251,3,0,0,1,7.171685,0,0
...,...,...,...,...,...,...,...,...
354,14.549459,11,0,0,22,14.532129,0,0
355,15.747258,14,0,0,22,15.159714,0,0
356,15.768089,17,0,0,22,15.432418,0,0
357,16.151361,13,0,0,22,15.466116,0,0


In [ ]:
def score_classification(model):
    score = 0
    for circuit in df[df.season == 2022]['round'].unique():

        test = df[(df.season == 2022) & (df['round'] == circuit)]
        X_test = test.drop(['driver', 'podium'], axis = 1)
        y_test = test.podium

        #scaling
        X_test = pd.DataFrame(scaler.transform(X_test), columns = X_test.columns)

        # make predictions
        prediction_df = pd.DataFrame(model.predict_proba(X_test), columns = ['proba_0', 'proba_1'])
        prediction_df['actual'] = y_test.reset_index(drop = True)
        prediction_df.sort_values('proba_1', ascending = False, inplace = True)
        prediction_df.reset_index(inplace = True, drop = True)
        prediction_df['predicted'] = prediction_df.index
        prediction_df['predicted'] = prediction_df.predicted.map(lambda x: 1 if x == 0 else 0)

        score += precision_score(prediction_df.actual, prediction_df.predicted)

    model_score = score / len(df[df.season == 2022]['round'].unique())
    return model_score, prediction_df

In [ ]:
# Neural network

# params={'hidden_layer_sizes': [(80,20,40,5), (75,25,50,10)], 
#         'activation': ['identity', 'logistic', 'tanh', 'relu'], 
#         'solver': ['lbfgs', 'sgd', 'adam', 'logistic'], 
#         'alpha': np.logspace(-4,2,20)} 


# for hidden_layer_sizes in params['hidden_layer_sizes']:
#     for activation in params['activation']:
#         for solver in params['solver']:
#             for alpha in params['alpha']:
#model_params = ( (75,25,50,10), 'identity', 'lbfgs', 0.01623776739188721)
model = MLPClassifier(hidden_layer_sizes = (75,25,50,10),
                      activation = 'identity', solver = 'lbfgs', alpha = 0.01623776739188721, random_state = 1)

model.fit(X_train, y_train)

score_nn = score_classification(model)
model_score_nn = score_nn[0]
preds_nn = score_nn[1]


# comparison_dict['model'].append('neural_network_classifier')
# comparison_dict['params'].append(model_params)
# comparison_dict['score'].append(model_score)

/usr/local/lib/python3.8/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


ValueError: ignored